In [1]:
import os
import random
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=str(5)

# Wide Residual Network obtained from github
## https://github.com/asmith26/wide_resnets_keras/blob/master/main.py

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from six.moves import range
import os

import logging
logging.basicConfig(level=logging.DEBUG)

import sys
sys.stdout = sys.stderr
# Prevent reaching to maximum recursion depth in `theano.tensor.grad`
sys.setrecursionlimit(2 ** 20)

import numpy as np
np.random.seed(2 ** 10)

from keras.datasets import cifar10
from keras.models import Model, load_model
from keras.layers import Input, Activation, merge, Dense, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
import keras



# ================================================
# DATA CONFIGURATION:
logging.debug("Loading data...")

nb_classes = 10
image_size = 32

from scipy.io import loadmat
train = loadmat('train.mat')
test = loadmat('test.mat')
X_train = train['X']
y_train = train['y']
X_test = test['X']
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train[np.newaxis,...]
X_train = np.swapaxes(X_train,0,4).squeeze()
X_test = X_test[np.newaxis,...]
X_test = np.swapaxes(X_test,0,4).squeeze()


np.place(y_train,y_train == 10,0)

y_train = keras.utils.to_categorical(y_train, 10)
# ================================================

# Create train and test for training model
Xtrain = X_train[1:65931]
Ytrain = y_train[1:65931]
Xtest = X_train[65931:]
Ytest = y_train[65931:]

# ================================================
# NETWORK/TRAINING CONFIGURATION:
logging.debug("Loading network/training configuration...")

depth = 34              # table 5 on page 8 indicates best value (4.17) CIFAR-10
k = 2                  # 'widen_factor'; table 5 on page 8 indicates best value (4.17) CIFAR-10
dropout_probability = 0.4 # table 6 on page 10 indicates best value (4.17) CIFAR-10

weight_decay = 0.0005   # page 10: "Used in all experiments"

batch_size = 128        # page 8: "Used in all experiments"
# Regarding nb_epochs, lr_schedule and sgd, see bottom page 10:
nb_epochs = 240
lr_schedule = [80, 160] # epoch_step
def schedule(epoch_idx):
    if (epoch_idx + 1) < lr_schedule[0]:
        return 0.01
    elif (epoch_idx + 1) < lr_schedule[1]:
        return 0.001 # lr_decay_ratio = 0.2
    return 0.0001
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)

# Other config from code; throughtout all layer:
use_bias = False        # following functions 'FCinit(model)' and 'DisableBias(model)' in utils.lua
weight_init="he_normal" # follows the 'MSRinit(model)' function in utils.lua

# Keras specific
if K.image_dim_ordering() == "th":
    logging.debug("image_dim_ordering = 'th'")
    channel_axis = 1
    input_shape = (3, image_size, image_size)
else:
    logging.debug("image_dim_ordering = 'tf'")
    channel_axis = -1
    input_shape = (image_size, image_size, 3)
# ================================================

# ================================================
# OUTPUT CONFIGURATION:
print_model_summary = True
save_model_and_weights = True
save_model_plot = False

MODEL_PATH = os.environ.get('MODEL_PATH', 'models/')
CHECKPOINT_PATH = os.environ.get('CHECKPOINT_PATH', 'checkpoints/')
# ================================================


# Wide residual network http://arxiv.org/abs/1605.07146
def _wide_basic(n_input_plane, n_output_plane, stride):
    def f(net):
        # format of conv_params:
        #               [ [nb_col="kernel width", nb_row="kernel height",
        #               subsample="(stride_vertical,stride_horizontal)",
        #               border_mode="same" or "valid"] ]
        # B(3,3): orignal <<basic>> block
        conv_params = [ [3,3,stride,"same"],
                        [3,3,(1,1),"same"] ]
        
        n_bottleneck_plane = n_output_plane

        # Residual block
        for i, v in enumerate(conv_params):
            if i == 0:
                if n_input_plane != n_output_plane:
                    net = BatchNormalization(axis=channel_axis)(net)
                    net = Activation("relu")(net)
                    convs = net
                else:
                    convs = BatchNormalization(axis=channel_axis)(net)
                    convs = Activation("relu")(convs)
                convs = Convolution2D(n_bottleneck_plane, nb_col=v[0], nb_row=v[1],
                                     subsample=v[2],
                                     border_mode=v[3],
                                     init=weight_init,
                                     W_regularizer=l2(weight_decay),
                                     bias=use_bias)(convs)
            else:
                convs = BatchNormalization(axis=channel_axis)(convs)
                convs = Activation("relu")(convs)
                if dropout_probability > 0:
                   convs = Dropout(dropout_probability)(convs)
                convs = Convolution2D(n_bottleneck_plane, nb_col=v[0], nb_row=v[1],
                                     subsample=v[2],
                                     border_mode=v[3],
                                     init=weight_init,
                                     W_regularizer=l2(weight_decay),
                                     bias=use_bias)(convs)

        # Shortcut Conntection: identity function or 1x1 convolutional
        #  (depends on difference between input & output shape - this
        #   corresponds to whether we are using the first block in each
        #   group; see _layer() ).
        if n_input_plane != n_output_plane:
            shortcut = Convolution2D(n_output_plane, nb_col=1, nb_row=1,
                                     subsample=stride,
                                     border_mode="same",
                                     init=weight_init,
                                     W_regularizer=l2(weight_decay),
                                     bias=use_bias)(net)
        else:
            shortcut = net

        return merge([convs, shortcut], mode="sum")
    
    return f


# "Stacking Residual Units on the same stage"
def _layer(block, n_input_plane, n_output_plane, count, stride):
    def f(net):
        net = block(n_input_plane, n_output_plane, stride)(net)
        for i in range(2,int(count+1)):
            net = block(n_output_plane, n_output_plane, stride=(1,1))(net)
        return net
    
    return f


def create_model():
    logging.debug("Creating model...")
    
    assert((depth - 4) % 6 == 0)
    n = (depth - 4) / 6
    
    inputs = Input(shape=input_shape)

    n_stages=[16, 16*k, 32*k, 64*k]

    conv1 = Convolution2D(nb_filter=n_stages[0], nb_row=3, nb_col=3, 
                          subsample=(1, 1),
                          border_mode="same",
                          init=weight_init,
                          W_regularizer=l2(weight_decay),
                          bias=use_bias)(inputs) # "One conv at the beginning (spatial size: 32x32)"

    # Add wide residual blocks
    block_fn = _wide_basic
    conv2 = _layer(block_fn, n_input_plane=n_stages[0], n_output_plane=n_stages[1], count=n, stride=(1,1))(conv1)# "Stage 1 (spatial size: 32x32)"
    conv3 = _layer(block_fn, n_input_plane=n_stages[1], n_output_plane=n_stages[2], count=n, stride=(2,2))(conv2)# "Stage 2 (spatial size: 16x16)"
    conv4 = _layer(block_fn, n_input_plane=n_stages[2], n_output_plane=n_stages[3], count=n, stride=(2,2))(conv3)# "Stage 3 (spatial size: 8x8)"

    batch_norm = BatchNormalization(axis=channel_axis)(conv4)
    relu = Activation("relu")(batch_norm)
                                            
    # Classifier block
    pool = AveragePooling2D(pool_size=(8, 8), strides=(1, 1), border_mode="same")(relu)
    flatten = Flatten()(pool)
    predictions = Dense(output_dim=nb_classes, init=weight_init, bias=use_bias,
                        W_regularizer=l2(weight_decay), activation="softmax")(flatten)

    model = Model(input=inputs, output=predictions)
    return model

Using TensorFlow backend.
DEBUG:root:Loading data...
DEBUG:root:Loading network/training configuration...
DEBUG:root:image_dim_ordering = 'tf'


In [3]:
model = create_model()
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=['accuracy'])

if print_model_summary:
    logging.debug("Model summary...")
    model.count_params()
    model.summary()
    
    
# Data Augmentation based on page 6 (see README for full details)
logging.debug("Creating ImageDataGenerators...")
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

datagen.fit(Xtrain, augment=True)


callbacks = [ LearningRateScheduler(schedule=schedule),
             ModelCheckpoint('weights.best.wrn_34-2.hdf5',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                            mode='auto')
            ]


logging.debug("Running training...")
# fit the model on the batches generated by train_datagen.flow()
model.fit_generator(datagen.flow(Xtrain, Ytrain, batch_size=batch_size, shuffle=True),
                    steps_per_epoch=Xtrain.shape[0] // batch_size,
                    epochs=nb_epochs,
                    validation_data=datagen.flow(Xtest, Ytest, batch_size=batch_size),
                    validation_steps=Xtest.shape[0] // batch_size,
                    callbacks=callbacks)

if save_model_and_weights:
    logging.debug("Saving model...")
    with open( os.path.join('WRN-{0}-{1}.json'.format(depth, k)), 'w') as f:
        f.write(model.to_json())
        model.save_weights( os.path.join(MODEL_PATH, 'WRN-{0}-{1}.h5'.format(depth, k)), overwrite=True)


DEBUG:root:Creating model...
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:200: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(use_bias=False, filters=16, kernel_size=(3, 3), kernel_initializer="he_normal", strides=(1, 1), kernel_regularizer=<keras.reg..., padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:142: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, kernel_initializer="he_normal", strides=(1, 1), kernel_regularizer=<keras.reg..., padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:153: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, kernel_initializer="he_normal", strides=(1, 1), kernel_regularizer=<keras.reg..., padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:165: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), use_bias=False, kernel_ini

DEBUG:root:Running training...
Epoch 1/240


KeyboardInterrupt: 

In [5]:
model = create_model()
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=['accuracy'])

if print_model_summary:
    logging.debug("Model summary...")
    model.count_params()
    model.summary()
    
    
# Data Augmentation based on page 6 (see README for full details)
logging.debug("Creating ImageDataGenerators...")
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

datagen.fit(Xtrain, augment=True)


callbacks = [ LearningRateScheduler(schedule=schedule),
             ModelCheckpoint('weights.best.wrn_34-2.hdf5',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                            mode='auto')
            ]

model.load_weights('weights.best.wrn_34-2.hdf5')
logging.debug("Running training...")
# fit the model on the batches generated by train_datagen.flow()
model.fit_generator(datagen.flow(Xtrain, Ytrain, batch_size=batch_size, shuffle=True),
                    steps_per_epoch=Xtrain.shape[0] // batch_size,
                    epochs=nb_epochs,
                    validation_data=datagen.flow(Xtest, Ytest, batch_size=batch_size),
                    validation_steps=Xtest.shape[0] // batch_size,
                    callbacks=callbacks)

if save_model_and_weights:
    logging.debug("Saving model...")
    with open( os.path.join('WRN-{0}-{1}.json'.format(depth, k)), 'w') as f:
        f.write(model.to_json())
        model.save_weights( os.path.join(MODEL_PATH, 'WRN-{0}-{1}.h5'.format(depth, k)), overwrite=True)

DEBUG:root:Creating model...
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:200: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(use_bias=False, filters=16, kernel_size=(3, 3), kernel_initializer="he_normal", strides=(1, 1), kernel_regularizer=<keras.reg..., padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:142: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, kernel_initializer="he_normal", strides=(1, 1), kernel_regularizer=<keras.reg..., padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:153: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, kernel_initializer="he_normal", strides=(1, 1), kernel_regularizer=<keras.reg..., padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:165: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), use_bias=False, kernel_ini

DEBUG:root:Running training...
Epoch 1/240
/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py:579: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py:571: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
 12/515 [..............................] - ETA: 4:37 - loss: 0.1787 - acc: 0.9772

KeyboardInterrupt: 

In [4]:
import pandas as pd

prob_y = model.predict_generator(datagen.flow(X_test,batch_size=128, shuffle=False), workers=16, use_multiprocessing=True)
pred_y = prob_y.argmax(axis=-1)
np.place(pred_y,pred_y == 0,10)
table = pd.DataFrame()
pred_id = list(range(1,X_test.shape[0]+1))
table.insert(0, 'id', pred_id)
table.insert(1, 'class', pred_y)
table.to_csv("WideResnet" + '_predictions.csv', sep=',', index=False)

/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py:571: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py:571: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py:571: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py:571: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it

/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py:579: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py:571: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py:579: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
